In [ ]:
!pip install tensorflow-addons


In [ ]:
!pip uninstall -y tensorflow keras tensorflow-addons
!pip install "tensorflow==2.15.0" "keras==2.15.0" "tensorflow-addons==0.23.0" opencv-python

In [ ]:
import os
import requests
import re
from tqdm import tqdm

# Configuración
set_code = "fdn"
base_dir = "mtg_foundations"
os.makedirs(base_dir, exist_ok=True)

def sanitize_name(name):
    """Crea nombres seguros para carpetas"""
    return re.sub(r'[^a-zA-Z0-9]', '_', name)[:50]

# Consultar API
url = f"https://api.scryfall.com/cards/search?q=e:{set_code}"
while url:
    response = requests.get(url)
    data = response.json()

    for card in tqdm(data.get('data', []), desc="Descargando"):
        card_name = card['name']
        safe_name = sanitize_name(card_name)
        class_dir = os.path.join(base_dir, safe_name)
        os.makedirs(class_dir, exist_ok=True)

        # Obtener URL de imagen
        image_url = card.get('image_uris', {}).get('png', '')
        if not image_url:
            continue

        # Generar nombre único
        file_ext = image_url.split('.')[-1].split('?')[0]
        file_name = f"{safe_name}_{card['collector_number']}.{file_ext}"
        file_path = os.path.join(class_dir, file_name)

        # Descargar solo si no existe
        if not os.path.exists(file_path):
            try:
                response = requests.get(image_url, stream=True, timeout=10)
                if response.status_code == 200:
                    with open(file_path, 'wb') as f:
                        for chunk in response.iter_content(1024):
                            f.write(chunk)
            except Exception as e:
                print(f"Error en {card_name}: {e}")

    url = data.get('next_page')

print(f"\n✅ Estructura creada en: {base_dir}")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications
import numpy as np
import cv2
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import tensorflow_addons as tfa

# Configuración
DATA_DIR = "/content/mtg_foundations"
AUG_DIR = "/content/augmented_dataset"
IMG_SIZE = (512, 512)
BATCH_SIZE = 16
EPOCHS_PER_GROUP = 10  # Reducir épocas para cumplir con el límite de tiempo
AUG_PER_CLASS = 1000  # Número de imágenes aumentadas por clase
NUM_THREADS = 8  # Número de hilos para multithreading
GROUP_SIZE = 44  # Número de clases por grupo

# 1. Aumentación de Datos Brutal
class CardAugmenter:
    def __init__(self, original_size=(600, 840)):
        self.original_size = original_size

    def random_perspective(self, img):
        pts1 = np.float32([[0,0],[600,0],[0,840],[600,840]])
        pts2 = np.float32([[np.random.randint(-50,50), np.random.randint(-50,50)],
                          [np.random.randint(550,650), np.random.randint(-50,50)],
                          [np.random.randint(-50,50), np.random.randint(790,890)],
                          [np.random.randint(550,650), np.random.randint(790,890)]])
        M = cv2.getPerspectiveTransform(pts1, pts2)
        return cv2.warpPerspective(img, M, self.original_size)

    def augment_image(self, img):
        img = self.random_perspective(img)
        img = tf.image.random_brightness(img, 0.4)
        img = tf.image.random_contrast(img, 0.5, 1.5)
        img = tf.image.random_hue(img, 0.08)
        img = tf.image.random_saturation(img, 0.6, 1.6)
        img = tf.image.random_jpeg_quality(img, 30, 100)
        img = tf.image.random_crop(img, size=[500, 500, 3])
        img = tf.image.resize_with_pad(img, *self.original_size)
        img = tf.image.resize_with_crop_or_pad(img, 600, 840)
        if np.random.rand() > 0.5:
            img = tfa.image.gaussian_filter2d(img, filter_shape=(3,3), sigma=1.0)
        return img.numpy()

# 2. Generar dataset aumentado para un grupo específico
def generate_augmented_dataset_for_group(class_dirs):
    augmenter = CardAugmenter()

    def process_image(class_dir, img, i):
        aug_img = augmenter.augment_image(img)
        cv2.imwrite(os.path.join(AUG_DIR, class_dir, f"aug_{i}.jpg"),
                    cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))

    for class_dir in tqdm(class_dirs, desc="Procesando clases"):
        orig_img_path = os.path.join(DATA_DIR, class_dir, os.listdir(os.path.join(DATA_DIR, class_dir))[0])
        img = cv2.imread(orig_img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        os.makedirs(os.path.join(AUG_DIR, class_dir), exist_ok=True)
        cv2.imwrite(os.path.join(AUG_DIR, class_dir, "original.jpg"), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

        with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
            futures = []
            for i in range(AUG_PER_CLASS):
                futures.append(executor.submit(process_image, class_dir, img, i))

            for future in tqdm(futures, desc=f"Aumentando {class_dir}", leave=False):
                future.result()

# 3. Dividir las clases en grupos
all_classes = sorted(os.listdir(DATA_DIR))
num_groups = len(all_classes) // GROUP_SIZE + (1 if len(all_classes) % GROUP_SIZE != 0 else 0)
groups = [all_classes[i * GROUP_SIZE:(i + 1) * GROUP_SIZE] for i in range(num_groups)]

# 4. Entrenar submodelos
submodels = []
for group_idx, group in enumerate(groups):
    print(f"\nEntrenando submodelo para el grupo {group_idx + 1}/{len(groups)}")

    # Generar datos aumentados para este grupo
    generate_augmented_dataset_for_group(group)

    # Cargar dataset aumentado
    train_ds = tf.keras.utils.image_dataset_from_directory(
        AUG_DIR,
        validation_split=0.2,
        subset="training",
        label_mode="categorical",
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        AUG_DIR,
        validation_split=0.2,
        subset="validation",
        label_mode="categorical",
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE
    )

    # Construir y entrenar el submodelo
    model = build_model(len(train_ds.class_names))
    model.compile(
        optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS_PER_GROUP
    )

    # Guardar el submodelo
    submodel_path = f"submodel_group_{group_idx + 1}.keras"
    model.save(submodel_path)
    submodels.append(model)

# 5. Combinar los submodelos en un modelo final
def combine_submodels(submodels, num_classes_total):
    inputs = layers.Input(shape=(*IMG_SIZE, 3))
    outputs = []

    for submodel in submodels:
        submodel.trainable = False  # Congelar los submodelos
        outputs.append(submodel(inputs))

    combined_output = layers.Concatenate()(outputs)
    final_output = layers.Dense(num_classes_total, activation="softmax")(combined_output)

    combined_model = models.Model(inputs=inputs, outputs=final_output)
    combined_model.compile(
        optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-5),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return combined_model

# Crear el modelo combinado
combined_model = combine_submodels(submodels, num_classes_total=len(all_classes))

# Entrenar el modelo combinado si es necesario
print("\nEntrenando el modelo combinado...")
train_ds_full = tf.keras.utils.image_dataset_from_directory(
    AUG_DIR,
    validation_split=0.2,
    subset="training",
    label_mode="categorical",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)
val_ds_full = tf.keras.utils.image_dataset_from_directory(
    AUG_DIR,
    validation_split=0.2,
    subset="validation",
    label_mode="categorical",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

combined_model.fit(
    train_ds_full,
    validation_data=val_ds_full,
    epochs=5  # Pocas épocas para ajustar el modelo combinado
)

# Guardar el modelo final
combined_model.save("final_combined_model.keras")
print("Modelo combinado guardado como 'final_combined_model.keras'")